In [25]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from sklearn.preprocessing import label_binarize
import numpy as np
import pandas as pd
from transformers import pipeline
from sklearn.feature_selection import f_classif
import warnings
warnings.filterwarnings('ignore')

# Load Dataset

In [26]:
df = pd.read_csv("final_data/val_dataset.csv") 
initial_df = df
df.head()

,review_id,user_id,user_name,time,rating,review,gmap_id,business_name,latitude,longitude,...,review_length_words,review_embedding,business_desc_embedding,name_validity,review_similarity,category_similarity,avg_review_rating_by_user,std_review_rating_by_user,posts_per_year,count_significantly_different_ratings
0,14,1.110000e+20,carlella rogers,2020-09-13 12:26:40,5.0,It was much more than we expected. The vows th...,0x80c8c3420f24ea4d:0x9955639d4bfb4fed,Love Story Wedding Chapel,36.162558,-115.145341,...,21,[-4.26931567e-02 1.31715611e-01 4.49305214e-...,[ 4.10041586e-02 3.15367468e-02 -1.20233428e-...,True,0.0,0.0,4.121429,1.334651,46.666667,175.0
1,28,1.110000e+20,Grant Spear,2020-09-13 12:26:40,5.0,The 3 for $10 deal was like fast food prices f...,0x80c8d01a85f6e1a9:0x5859654d3e9f6ff,Chili's Grill & Bar,36.006249,-115.112255,...,14,[-5.30603677e-02 -4.31169989e-03 1.87707786e-...,[-9.25057009e-03 5.52590098e-03 5.15403524e-...,True,0.0,0.0,4.121429,1.334651,46.666667,175.0
2,34,1.160000e+20,Scott Burley,2021-05-03 00:00:00,5.0,I am not one of those people from Louisiana th...,0x80c8eb682be17b6d:0x9c61c8c4c5915636,DB's Cajun Kitchen,36.238434,-115.232408,...,32,[-3.19478102e-02 -2.96392664e-02 3.22441459e-...,[-5.73068997e-03 -7.18563935e-03 5.80819091e-...,True,0.0,0.0,4.241319,1.303812,44.307692,165.0
3,37,1.110000e+20,Darcie Jenkins,2021-01-07 06:13:20,4.0,Ordered delivery through Postmates. We ordered...,0x80c8c124d00b4e2b:0xc42765dc240d7f8c,New York Pizza & Pasta,36.145168,-115.224505,...,116,[-7.39922822e-02 1.07164048e-02 2.27621701e-...,[-3.54432911e-02 7.32429773e-02 2.95847449e-...,False,0.0,0.0,4.121429,1.334651,46.666667,175.0
4,39,1.160000e+20,Lynne Grill,2015-12-13 09:46:40,4.0,Very well appointed rooms friendly and helpful...,0x80c8c430cb5147bd:0x7739e012bcbfe8e4,Club Wyndham Grand Desert,36.107183,-115.161430,...,23,[ 8.71474147e-02 -1.99438892e-02 6.91945404e-...,[ 5.75988032e-02 1.89542416e-02 -8.17048550e-...,True,0.0,0.0,4.241319,1.303812,44.307692,165.0


In [27]:
df['label_rules'] = "relevant"

In [28]:
def detect_ads_leave_as_is(row):
    text = row['review']
    current_label = row['label_rules']
    
    if pd.isna(text):
        return current_label  # leave as is
    
    # Check for URLs
    if re.search(r'http\S+|www\S+|https\S+', text, flags=re.MULTILINE):
        return "ads"
    # Check for phone numbers
    if re.search(r'\+?\d[\d\-\(\) ]{7,}\d', text):
        return "ads"
    
    return current_label  # leave as is if not detected

# Apply function
df['label_rules'] = df.apply(detect_ads_leave_as_is, axis=1)

# Quick check
print(df[['review', 'label_rules']].head())

                                              review label_rules
0  It was much more than we expected. The vows th...    relevant
1  The 3 for $10 deal was like fast food prices f...    relevant
2  I am not one of those people from Louisiana th...    relevant
3  Ordered delivery through Postmates. We ordered...    relevant
4  Very well appointed rooms friendly and helpful...    relevant


In [29]:
# Detect rants

# Define strong rant indicators
RANT_PATTERNS = [
    r"\bnever been\b",
    r"\bnever visited\b",
    r"\bnot been\b",
    r"\bhavent visited\b",
    r"\bdon'?t know (this|the) place\b",
    r"\bdon'?t know what (they|this) do(es)?\b",
    r"\bheard about this place\b"
]
rant_re = re.compile("|".join(RANT_PATTERNS), re.I)

# Define exceptions (phrases that imply they *have* visited)
RANT_EXCEPTIONS = [
    "in a while",
    "again",
    "before",
    "recently",
    "so decided to check it out",
    "so i went",
    "so i tried",
    "so i checked"
]

def is_rant(text: str) -> bool:
    if not text or not isinstance(text, str):
        return False
    
    t = text.lower()

    # Step 1: does it contain any rant trigger?
    if not rant_re.search(t):
        return False

    # Step 2: check if any exception is present
    for exc in RANT_EXCEPTIONS:
        if exc in t:
            return False

    # If we found a trigger and no exception => classify as rant
    return True

df["is_rant"] = df["review"].apply(is_rant)

df.loc[(df["is_rant"]) & (df["label_rules"] == "relevant"), "label_rules"] = "rant"

df[df["is_rant"] & (df["label"] == "relevant")]["review"]

20     Not my first rodeo in Las Vegas. Caesars Palac...
239    Heard about this place from a podcast called t...
282    I love Front Sight. My wife and I have been me...
444    (Translated by Google) Pity customer service. ...
548    I have been going to V&V for about three years...
650       It was truly fantastic from a world never Been
Name: review, dtype: object

In [30]:
mask = (df["label_rules"] == "relevant") & (df["sentiment_label"] == "Negative")
negative_relevant = df[mask].copy()

# Step 2: Compute 50th percentile of sentiment_score for this subset
score_threshold = negative_relevant["sentiment_score"].quantile(0.5)

# Step 3: Label top 50% as 'rant'
negative_relevant.loc[negative_relevant["sentiment_score"] > score_threshold, "label_rules"] = "rant"

# Step 4: Update original DataFrame
df.update(negative_relevant)

In [31]:
# top 10% most repetitive
threshold = df['repetitiveness_score'].quantile(0.90)
df.loc[(df['repetitiveness_score'] > threshold)&(df['label_rules']=='relevant') , 'label_rules'] = 'spam'

# check results
print(df[['review', 'repetitiveness_score', 'label_rules']].head(10))


                                              review  repetitiveness_score  \
0  It was much more than we expected. The vows th...              0.091695   
1  The 3 for $10 deal was like fast food prices f...              0.092533   
2  I am not one of those people from Louisiana th...              0.000000   
3  Ordered delivery through Postmates. We ordered...              0.074770   
4  Very well appointed rooms friendly and helpful...              0.071530   
5  The best place for karaoke. The owners are alw...              0.109548   
6  Above average price and below average quality!...              0.018978   
7                               Always a good time!!              0.170324   
8  My experience was awesome. The entire staff wa...              0.018410   
9                         I've had better corn dogs.              0.000000   

  label_rules  
0    relevant  
1    relevant  
2    relevant  
3    relevant  
4    relevant  
5    relevant  
6    relevant  
7    relevant

In [32]:
emoji_pattern = re.compile(
    r'^[\U0001F600-\U0001F64F'  # emoticons
    r'\U0001F300-\U0001F5FF'  # symbols & pictographs
    r'\U0001F680-\U0001F6FF'  # transport & map symbols
    r'\U0001F1E0-\U0001F1FF'  # flags
    r'\U00002700-\U000027BF'  # dingbats
    r'\U0001F900-\U0001F9FF'  # supplemental symbols & pictographs
    r'\U00002600-\U000026FF'  # miscellaneous symbols
    r'\U00002B00-\U00002BFF'  # miscellaneous symbols & arrows
    r'\U0000200D'              # zero-width joiner
    r'\U000024C2-\U0001F251]+$'
)

# Function to check if review has only emojis
def is_emoji_only(text):
    if pd.isna(text):
        return False
    text = text.strip()
    return bool(emoji_pattern.fullmatch(text))

# Update label to 'irrelevant' if review is only emojis
df.loc[(df['review'].apply(is_emoji_only))&(df['label_rules']=='relevant'), 'label_rules'] = 'irrelevant'

# Quick check
print(df[['review', 'label_rules']].head(10))

                                              review label_rules
0  It was much more than we expected. The vows th...    relevant
1  The 3 for $10 deal was like fast food prices f...    relevant
2  I am not one of those people from Louisiana th...    relevant
3  Ordered delivery through Postmates. We ordered...    relevant
4  Very well appointed rooms friendly and helpful...    relevant
5  The best place for karaoke. The owners are alw...    relevant
6  Above average price and below average quality!...    relevant
7                               Always a good time!!    relevant
8  My experience was awesome. The entire staff wa...    relevant
9                         I've had better corn dogs.        rant


In [33]:
import numpy as np

def parse_embedding(s):
    # Remove brackets and turn into numpy array of floats
    return np.fromstring(s.strip("[]"), sep=" ")

# Apply parsing
df["business_desc_embedding"] = df["business_desc_embedding"].apply(parse_embedding)
df["review_embedding"] = df["review_embedding"].apply(parse_embedding)

# Cosine similarity function
def cosine_sim(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# Compute cosine similarity
df["cosine_similarity"] = df.apply(
    lambda row: cosine_sim(row["business_desc_embedding"], row["review_embedding"]),
    axis=1
)

print(df[["business_desc_embedding", "review_embedding", "cosine_similarity"]].head())

                             business_desc_embedding  \
0  [0.0410041586, 0.0315367468, -0.0120233428, 0....   
1  [-0.00925057009, 0.00552590098, 0.0515403524, ...   
2  [-0.00573068997, -0.00718563935, 0.00580819091...   
3  [-0.0354432911, 0.0732429773, 0.0295847449, 0....   
4  [0.0575988032, 0.0189542416, -0.081704855, 0.0...   

                                    review_embedding  cosine_similarity  
0  [-0.0426931567, 0.131715611, 0.0449305214, -0....           0.364528  
1  [-0.0530603677, -0.00431169989, 0.0187707786, ...           0.235333  
2  [-0.0319478102, -0.0296392664, 0.0322441459, 0...           0.411910  
3  [-0.0739922822, 0.0107164048, 0.0227621701, 0....           0.536538  
4  [0.0871474147, -0.0199438892, 0.0691945404, 0....           0.312256  


In [34]:
# Filter rows where label_rules is 'relevant' and cosine_similarity < 0.23
mask = (df["label_rules"] == "relevant") & (df["cosine_similarity"] < 0.05)

# Replace label with 'irrelevant' for those rows
df.loc[mask, "label_rules"] = "irrelevant"

# Check changes
df[mask][["label_rules", "cosine_similarity", "label"]].head()

,label_rules,cosine_similarity,label
46,irrelevant,0.016778,relevant
131,irrelevant,0.047573,relevant
160,irrelevant,0.032833,relevant
170,irrelevant,-0.014728,relevant
180,irrelevant,0.003868,irrelevant


In [35]:
mask = (df["label_rules"] == "relevant") & (df["sentiment_label"] == "Negative")
negative_relevant = df[mask].copy()

# Step 2: Compute 50th percentile of sentiment_score for this subset
score_threshold = negative_relevant["sentiment_score"].quantile(0.5)

# Step 3: Label top 50% as 'rant'
negative_relevant.loc[negative_relevant["sentiment_score"] < score_threshold, "label_rules"] = "irrelevant"

# Step 4: Update original DataFrame
df.update(negative_relevant)

In [36]:
match = df['label'] == df['label_rules']

# Count how many match
print(match.sum(), "rows match")

# Count how many differ
print((~match).sum(), "rows differ")

1551 rows match
438 rows differ


In [37]:
df_diff = df[df['label'] != df['label_rules']]
print(df_diff[['label', 'label_rules']])

           label label_rules
9       relevant        rant
16      relevant        rant
19      relevant        rant
20      relevant        rant
21      relevant        spam
...          ...         ...
1982  irrelevant        rant
1984  irrelevant    relevant
1985  irrelevant    relevant
1986  irrelevant    relevant
1987  irrelevant        spam

[438 rows x 2 columns]


In [38]:
# Boolean series: True if the labels are the same
match = df['label'] == df['label_rules']

# Percentage of agreement
agreement_pct = match.mean() * 100
print(f"Percentage of agreement: {agreement_pct:.2f}%")

Percentage of agreement: 77.98%


In [39]:
df[['label', 'label_rules']]

,label,label_rules
0,relevant,relevant
1,relevant,relevant
2,relevant,relevant
3,relevant,relevant
4,relevant,relevant
...,...,...
1984,irrelevant,relevant
1985,irrelevant,relevant
1986,irrelevant,relevant
1987,irrelevant,spam


In [40]:
# Get the ground truth and predicted labels
y_true = df['label']
y_pred = df['label_rules']

# --- Confusion matrix ---
cm = confusion_matrix(y_true, y_pred, labels=np.unique(y_true))
print("Confusion Matrix:")
print(pd.DataFrame(cm, index=np.unique(y_true), columns=np.unique(y_true)))

# --- Classification report: F1, precision, recall ---
print("\nClassification Report:")
print(classification_report(y_true, y_pred, digits=4))

# --- Multi-class ROC-AUC ---
# Binarize the labels for multi-class AUC
classes = np.unique(y_true)
y_true_bin = label_binarize(y_true, classes=classes)
y_pred_bin = label_binarize(y_pred, classes=classes)

# Compute macro-averaged ROC-AUC
roc_auc = roc_auc_score(y_true_bin, y_pred_bin, average='macro')
print(f"\nMacro-averaged ROC-AUC: {roc_auc:.4f}")



Confusion Matrix:
            ads  irrelevant  rant  relevant  spam
ads         249           0     0         2     0
irrelevant    0         105    29       121    18
rant          0           9   221        23     2
relevant      0          83   134       870    17
spam          0           0     0         0   106

Classification Report:
              precision    recall  f1-score   support

         ads     1.0000    0.9920    0.9960       251
  irrelevant     0.5330    0.3846    0.4468       273
        rant     0.5755    0.8667    0.6917       255
    relevant     0.8563    0.7880    0.8208      1104
        spam     0.7413    1.0000    0.8514       106

    accuracy                         0.7798      1989
   macro avg     0.7412    0.8063    0.7613      1989
weighted avg     0.7879    0.7798    0.7766      1989


Macro-averaged ROC-AUC: 0.8699


# Output as `review_id`, `true_label`, `pred_label`

In [41]:
final = df[['review_id', 'label', 'label_rules']].copy()
final = final.rename(columns={'label_rules': 'pred', 'label': 'true_label'})

# One-hot encode predicted_label with 0/1 instead of True/False
one_hot = pd.get_dummies(final['pred'], dtype=int)

# Concatenate back
final = pd.concat([final, one_hot], axis=1)

# Save
final.to_csv("results/rule_based.csv", index=False)